In [5]:
'''
!pip install pytorch-pretrained-bert
!pip install Unidecode
'''

'\n!pip install pytorch-pretrained-bert\n!pip install Unidecode\n'

In [6]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from collections import OrderedDict
import unidecode
import numpy as np
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)
import os
import matplotlib.pyplot as plt
# % matplotlib inline

In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Laptop GPU'

#### Import pre trained model 

In [7]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#Use the pre-trained Base BERT model 
model = BertModel.from_pretrained('bert-base-uncased')
model.cuda()
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Lin

In [8]:
import os

In [9]:
root_dir = 'C:\\Users\\Tsaro\\OneDrive\\Desktop\\NCSR Demokritos Data Science\\Thesis\\Datasets\\SemEVAL 2020'

In [10]:
root_dir

'C:\\Users\\Tsaro\\OneDrive\\Desktop\\NCSR Demokritos Data Science\\Thesis\\Datasets\\SemEVAL 2020'

##### Convert to zip 

In [22]:
import gzip

with gzip.open('ccoha1.txt.gz', 'rb') as fin:
    with open('ccoha1.txt', 'wb') as fout:
        data = fin.read()
        fout.write(data)
    

In [23]:
with gzip.open('ccoha2.txt.gz', 'rb') as fin:
    with open('ccoha2.txt', 'wb') as fout:
        data = fin.read()
        fout.write(data)
    

#### Import the data 

In [11]:
class Data():

    def __getitem__(self, path=None):
         if path!=None:
            with open(path,'r') as f:
                 self.doc=f.read()
            return self.doc
     
    def _preprocess(self,targets,corpus):
        self.index=[]
        self.t_index=OrderedDict()
        for target in targets:
            
            for _,item in enumerate(corpus):
                if target in item:

                      count_target=item.count(target)
                  #   Avoiding the sentences with multiple occurrences of the target term for the time being###
                      if count_target==1:
                        if target not in self.t_index.keys():
                            self.t_index[target]=[_]
                        else:
                            self.t_index[target].append(_)
                        self.index.append(_)
        return self.index,self.t_index

In [32]:
'''
LOAD & EXTRACT DATA

'''

p1 = os.path.join(root_dir, 'ccoha1.txt')
p2 = os.path.join(root_dir, 'ccoha2.txt')
t = os.path.join(root_dir, 'targets.txt')


datasets = Data() # initialization
doc1=datasets.__getitem__(p1).split('\n')   
doc2=datasets.__getitem__(p2).split('\n')
t1=datasets.__getitem__(t).split('\n')
target_act=[x for x in t1 if len(x)>1]
t1=[x.lower() for x in t1 if len(x)>1]
index1=datasets._preprocess(t1,doc1)
index2=datasets._preprocess(t1,doc2)         
index_t1=index1[1]
index_t2=index2[1]
print('The target words are:',t1)
target_words=t1



#conversions
target_uni=[unidecode.unidecode(m) for m in t1]
target_toks=[]


# print(target_uni)
for k in t1:
    target_toks.append(tokenizer.tokenize(k))

print(" ")
print('converted target toks',target_toks)


The target words are: ['attack_nn', 'bag_nn', 'ball_nn', 'bit_nn', 'chairman_nn', 'circle_vb', 'contemplation_nn', 'donkey_nn', 'edge_nn', 'face_nn', 'fiction_nn', 'gas_nn', 'graft_nn', 'head_nn', 'land_nn', 'lane_nn', 'lass_nn', 'multitude_nn', 'ounce_nn', 'part_nn', 'pin_vb', 'plane_nn', 'player_nn', 'prop_nn', 'quilt_nn', 'rag_nn', 'record_nn', 'relationship_nn', 'risk_nn', 'savage_nn', 'stab_nn', 'stroke_vb', 'thump_nn', 'tip_vb', 'tree_nn', 'twist_nn', 'word_nn']
 
converted target toks [['attack', '_', 'n', '##n'], ['bag', '_', 'n', '##n'], ['ball', '_', 'n', '##n'], ['bit', '_', 'n', '##n'], ['chairman', '_', 'n', '##n'], ['circle', '_', 'v', '##b'], ['con', '##tem', '##pl', '##ation', '_', 'n', '##n'], ['donkey', '_', 'n', '##n'], ['edge', '_', 'n', '##n'], ['face', '_', 'n', '##n'], ['fiction', '_', 'n', '##n'], ['gas', '_', 'n', '##n'], ['graf', '##t', '_', 'n', '##n'], ['head', '_', 'n', '##n'], ['land', '_', 'n', '##n'], ['lane', '_', 'n', '##n'], ['lass', '_', 'n', '##n'],

In [24]:
len(doc1)

253645

In [25]:
doc1

['he may find himself unexpectedly throw into new and strange position where after look carefully around him he fear to move lest by a mistake step he should do an injury to the cause which he have it in his heart to serve',
 'first and foremost he draw forth from a case in the corner of his lodging a brass-buttoned blue coat of a popular cut and fall to beat it over the shoulder and down the back with a yard stick as if he have under his hand the body and person of his dire enemy in the world then he twist the right arm up and dash at the place where the rib might have be then he fall upon the breast and pumelled them horribly and then cast aside his stick he fasten fiercely on the collar and give the whole a mighty shake as if he would have the very life out of it',
 'it be in the latter city that his great discovery be first make public or rather it be there that he be first suspect of have make it',
 'the farmer say he would wash himself first then wait on her immediately and prese

In [18]:
t1

['attack_nn',
 'bag_nn',
 'ball_nn',
 'bit_nn',
 'chairman_nn',
 'circle_vb',
 'contemplation_nn',
 'donkey_nn',
 'edge_nn',
 'face_nn',
 'fiction_nn',
 'gas_nn',
 'graft_nn',
 'head_nn',
 'land_nn',
 'lane_nn',
 'lass_nn',
 'multitude_nn',
 'ounce_nn',
 'part_nn',
 'pin_vb',
 'plane_nn',
 'player_nn',
 'prop_nn',
 'quilt_nn',
 'rag_nn',
 'record_nn',
 'relationship_nn',
 'risk_nn',
 'savage_nn',
 'stab_nn',
 'stroke_vb',
 'thump_nn',
 'tip_vb',
 'tree_nn',
 'twist_nn',
 'word_nn']

In [19]:
index1

([621,
  1322,
  1872,
  2079,
  4175,
  4390,
  5583,
  5799,
  6297,
  7340,
  7395,
  8534,
  9325,
  9805,
  10399,
  10633,
  11490,
  11783,
  12401,
  13184,
  14143,
  15236,
  15997,
  16410,
  17046,
  17297,
  18173,
  18556,
  19361,
  20163,
  20448,
  20587,
  20596,
  20737,
  21749,
  22402,
  22952,
  23350,
  23931,
  24712,
  26001,
  26170,
  27380,
  27499,
  28850,
  28873,
  28967,
  29335,
  29380,
  29740,
  31924,
  32690,
  32798,
  33337,
  34817,
  35151,
  35978,
  36498,
  37409,
  37840,
  38999,
  40839,
  41358,
  41448,
  41984,
  43349,
  44415,
  45021,
  46214,
  46392,
  46709,
  47184,
  47616,
  47671,
  47807,
  47999,
  48136,
  48208,
  48466,
  50406,
  50576,
  51352,
  51611,
  52210,
  53825,
  54303,
  54613,
  55901,
  56206,
  56706,
  57207,
  57677,
  58760,
  58997,
  59193,
  59592,
  60397,
  61093,
  61351,
  62134,
  62369,
  62860,
  63227,
  63232,
  63290,
  63438,
  63568,
  63837,
  64448,
  65364,
  65470,
  65827,
  67695

In [23]:
index_t1

OrderedDict([('attack_nn',
              [621,
               1322,
               1872,
               2079,
               4175,
               4390,
               5583,
               5799,
               6297,
               7340,
               7395,
               8534,
               9325,
               9805,
               10399,
               10633,
               11490,
               11783,
               12401,
               13184,
               14143,
               15236,
               15997,
               16410,
               17046,
               17297,
               18173,
               18556,
               19361,
               20163,
               20448,
               20587,
               20596,
               20737,
               21749,
               22402,
               22952,
               23350,
               23931,
               24712,
               26001,
               26170,
               27380,
               27499,
               28850

#####  BERT functions 

In [16]:
def _pre_bert(doc,index,t):
  
  s=[doc[ind] for ind in index[t]]
  print('len of sentences',len(s))
  l=len(s)
  marked_text = ["[CLS] " + text + " [SEP]" for text in s]
  tokenized_text = [tokenizer.tokenize(m) for m in marked_text]
  
  tokenized_text=[x[:512] if len(x)>512 else x for x in tokenized_text]
  indexed_tokens = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_text]
  segments_ids = [[1] * len(x) for x in tokenized_text]
  return s,marked_text,tokenized_text,indexed_tokens,segments_ids,l

In [26]:

def _bert_features(tokens_tensor, segments_tensors,tokenized_text):
  # print(len(tokens_tensor[0]))
  with torch.no_grad():
      encoded_layers, _ = model(tokens_tensor.to(device), segments_tensors.to(device))
  # print ("Number of layers:", len(encoded_layers))
  layer_i = 0

  # # print ("Number of batches:", len(encoded_layers[layer_i]))
  batch_i = 0

  # print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
  token_i = 0

  # print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))
  # Convert the hidden state embeddings into single token vectors

  # Holds the list of 12 layer embeddings for each token
  # Will have the shape: [# tokens, # layers, # features]
  token_embeddings = [] 

  # For each token in the sentence...
  # tokenized_text=[x for x in tokenized_text if x not in ['_', 'n', '##n','v', '##b']]
  for token_i in range(len(tokenized_text)):
    
    # Holds 12 layers of hidden states for each token 
    hidden_layers = [] 
    
    # For each of the 12 layers...
    for layer_i in range(len(encoded_layers)):
      
      # Lookup the vector for `token_i` in `layer_i`
      vec = encoded_layers[layer_i][batch_i][token_i]
      
      hidden_layers.append(vec)
      
    token_embeddings.append(hidden_layers)

  # Sanity check the dimensions:
  # print ("Number of tokens in sequence:", len(token_embeddings))
  # print ("Number of layers per token:", len(token_embeddings[0]))
  return token_embeddings
# s,marked_text,tokenized_text,indexed_tokens,segments_ids

In [27]:
def _get_embeddings(pre,tg):
  m_embed_full=[]
  # print('len(pre[0])',len(pre[0]))
  # print(tg)
  for _,item in enumerate(pre[0]):
    # Convert inputs to PyTorch tensors
    # print(item)
    token_list=pre[2][_]
    
    tokens_tensor = torch.tensor([pre[3][_]])
    segments_tensors = torch.tensor([pre[4][_]])
    
    # Predict hidden states features for each layer
    token_embeddings=_bert_features(tokens_tensor, segments_tensors,pre[2][_])
    concatenated_last_4_layers = [torch.cat((layer[-1], layer[-2], layer[-3], layer[-4]), 0) for layer in token_embeddings] # [number_of_tokens, 3072]

    summed_last_4_layers = [torch.sum(torch.stack(layer)[-4:], 0) for layer in token_embeddings] # [number_of_tokens, 768]
    
    #consider the tokenized target
  
    indxs=[]
    # print(token_list)
    for tok in tg:
      '''
      remove -1,-2,-3
      '''
      if tok in token_list:
        if tok not in ['_', 'n', '##n','v', '##b']:
          indxs.append(token_list.index(tok))

    # print('indxs',indxs)
    if len(indxs)==1:
      bert_embed=concatenated_last_4_layers [indxs[0]]
      m_embed_full.append(bert_embed)
    elif len(indxs)>1:
      b_emb=[]
      for ind in indxs:
        b_emb.append(concatenated_last_4_layers[ind])
      bert_embed= torch.sum(torch.stack(b_emb), 0)
      m_embed_full.append(bert_embed)
    # indx=token_list.index(tg.lower())
    # indx = [i for (i, elem) in enumerate(pre[2][_]) if t in elem]
    # print('indx',indx)
    # print(pre[1][_],indx)

    # if len(indx)>0:
    # bert_embed=concatenated_last_4_layers[indx[0]]
    
    # cosine_similarity(summed_last_4_layers[10].reshape(1,-1), summed_last_4_layers[19].reshape(1,-1))[0][0]
    
    
  return  m_embed_full


#### For a particular target word,do clustering and find if there is a sense change


In [28]:
#[target_words[1]]
def embeddings_extract(target_words,target_toks,doc1,index_t1,doc2,index_t2):
  t=target_words
  X=[]
  X_C1=[]
  X_C2=[]
  sents_all=[]
  lens1=[]
  lens2=[]
  for k,t in enumerate(target_words) :
    berts=[]
    sents=[]
    print('The target word is',t)    
    
    #get the sentences from corpus c1 and c2 for the specific target word 't'
    
    pre1=_pre_bert(doc1,index_t1,t)

    pre2=_pre_bert(doc2,index_t2,t)
    # lens1.append(pre1[-1])
    # lens2.append(pre2[-1])
    # print(pre1)
    
    sents.extend(pre1[0])
    sents.extend(pre2[0])
    #aggregate all the embeddings
    # s,marked_text,tokenized_text,indexed_tokens,segments_ids

    '''
    Get the embeddings of the targets from corpus 1 and 2
    '''
    b1=_get_embeddings(pre1,target_toks[k])
    print('len of t1',len(b1))
    b2=_get_embeddings(pre2,target_toks[k])
    print('len of t2',len(b2))
    '''
    store the lenghts of no. of sentences extracted for each target word for each corpus
    '''
    lens1.append(len(b1))
    lens2.append(len(b2))
    
    berts.extend(b1)
    berts.extend(b2)
    print('len of each target word extractions is',len(berts))
    X.append(berts)
    X_C1.append(b1)# the embeddings for C1
    X_C2.append(b2)#embeddings for C2
    sents_all.append(sents)
  return X,X_C1,X_C2,lens1,lens2,sents_all

In [38]:
X,X_C1,X_C2,lens1,lens2,sents_all = embeddings_extract(target_words,target_toks,doc1,index_t1,doc2,index_t2) 

The target word is attack_nn
len of sentences 440
len of sentences 803
len of t1 440
len of t2 803
len of each target word extractions is 1243
The target word is bag_nn
len of sentences 190
len of sentences 838
len of t1 190
len of t2 838
len of each target word extractions is 1028
The target word is ball_nn
len of sentences 408
len of sentences 791
len of t1 408
len of t2 791
len of each target word extractions is 1199
The target word is bit_nn
len of sentences 280
len of sentences 600
len of t1 280
len of t2 600
len of each target word extractions is 880
The target word is chairman_nn
len of sentences 133
len of sentences 636
len of t1 133
len of t2 636
len of each target word extractions is 769
The target word is circle_vb
len of sentences 131
len of sentences 235
len of t1 131
len of t2 235
len of each target word extractions is 366
The target word is contemplation_nn
len of sentences 240
len of sentences 108
len of t1 240
len of t2 108
len of each target word extractions is 348
Th